# GLDAS

In [ ]:
!pip install earthengine-api
!pip install geemap
!pip install pandas
!pip install matplotlib

In [ ]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the region of interest
ET_region = ee.Geometry.Point([29.25197, -28.9936])

# Load the GLDAS image collection
collection = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")

# Calculate the date range
range = collection.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])
diff = ee.Date(range.get('max')).difference(ee.Date(range.get('min')), 'months')

# Function to calculate monthly sums
def monthly_sum(n):
    start = ee.Date(range.get('min')).advance(n, 'month')
    end = start.advance(1, 'month')
    return collection \
            .filterDate(start, end) \
            .sum().multiply(24 * 60 * 60) \
            .set('system:time_start', start)

# Apply the function to get monthly sums
month_mean = ee.List.sequence(0, diff).map(monthly_sum)
ET = ee.ImageCollection(month_mean).select('Evap_tavg')

# Extract data for the region
ET_data = ET.getRegion(ET_region, 27830).getInfo()

# Convert to DataFrame for easier handling
df = pd.DataFrame(ET_data[1:], columns=ET_data[0])
df['datetime'] = pd.to_datetime(df['system:time_start'], unit='ms')
df['Evap_tavg'] = pd.to_numeric(df['Evap_tavg'])

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(df['datetime'], df['Evap_tavg'], marker='o', linestyle='-', markersize=4)
plt.title('ET GLDAS')
plt.xlabel('Date')
plt.ylabel('Average mm/day')
plt.grid(True)
plt.show()